# Assignment segment and cluster Boroughs of Toronto - part 3: Explore using Foursquare

## Matthias Reumann

In [1]:
# Install the lxml package in the current Jupyter kernel using conda
#import sys
#!conda install --yes --prefix {sys.prefix} lxml

There seem to be issues in installing the lxml package using conda in this environment. 
Edit: The issue seems to be resolved and the results are shown. When running the code locally in a python script, the code executed perfectly.

## Import the libraries

In [28]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get the table of Torontos neighbourhoods from the wikipedia page

In [29]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url, header=0)
print(tables[0].columns.values)

df = tables[0]
dfToronto = tables[0]
print(dfToronto.head(10))
print(dfToronto.shape)




['Postcode' 'Borough' 'Neighbourhood']
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned
(288, 3)


The table of Boroughs and neighbourhoods has 288 rows.

## Drop 'not assigned' fields in 'Borough'


In [30]:
# drop 'not assigned' fields in 'Borough'
# Get names of indexes
index = dfToronto[dfToronto['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
dfToronto.drop(index, inplace=True)
print(dfToronto.shape)
dfToronto.head(10)


(211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


After dropping the 'not assigned' rows in the Borough column, the table has 211 rows.

## Assign Neighbourhood 'Not assigned' to Borough name

In [31]:
# assign Neighbourhood 'Not assigned' to Borough name

index = dfToronto[dfToronto['Neighbourhood'] == 'Not assigned'].index

for i in index:
    print(i)
    dfToronto.loc[i]['Neighbourhood'] = dfToronto.loc[i]['Borough']

print(dfToronto.head(10))
print(dfToronto.shape)


8
   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Queen's Park
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern
(211, 3)


## Groupby Boroughs keeping the Neighbourhood as lists
Also, to keep a unique post code per neighbourhood, the grouping is done by Borough then Postcode. This differs from part 1 of the assignment.


In [32]:
# groupby Boroughs keeping the Neighbourhood as lists
#dfToronto = dfToronto.groupby(['Borough']).agg({'Postcode': list, 'Neighbourhood': list}).reset_index()
dfToronto = dfToronto.groupby(['Borough', 'Postcode']).agg({'Neighbourhood': list}).reset_index()

print(dfToronto.columns.values)
dfToronto = dfToronto[['Postcode', 'Borough', 'Neighbourhood']]
print(dfToronto.shape)
dfToronto

['Borough' 'Postcode' 'Neighbourhood']
(103, 3)


,Postcode,Borough,Neighbourhood
0,M4N,Central Toronto,[Lawrence Park]
1,M4P,Central Toronto,[Davisville North]
2,M4R,Central Toronto,[North Toronto West]
3,M4S,Central Toronto,[Davisville]
4,M4T,Central Toronto,"[Moore Park, Summerhill East]"
5,M4V,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H..."
6,M5N,Central Toronto,[Roselawn]
7,M5P,Central Toronto,"[Forest Hill North, Forest Hill West]"
8,M5R,Central Toronto,"[The Annex, North Midtown, Yorkville]"
9,M4W,Downtown Toronto,[Rosedale]


Toronto seems to have 11 Boroughs after grouping them. There are 103 Neighbourhoods.

## Add geo data to the dataframe by using the .csv file.

In [33]:
c=pd.read_csv("Geospatial_Coordinates.csv")

print(c.head())

dfToronto['Latitude'] = 0.0
dfToronto['Longitude'] = 0.0

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Latitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Latitude']
    dfToronto.at[index, 'Longitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Longitude']


print(dfToronto.columns.values)
print(dfToronto.shape)
dfToronto.head()


  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
['Postcode' 'Borough' 'Neighbourhood' 'Latitude' 'Longitude']
(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
1,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
2,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
3,M4S,Central Toronto,[Davisville],43.704324,-79.388790
4,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160


## Explore the Toronto neighbourhoods similar to the lab on exploring neighbourhoods of NYC

### Plot the neighbourhoods using folium

In [34]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [35]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Looking just at Central Toronto for now

In [36]:
CentralToronto_data = dfToronto[dfToronto['Borough'] == 'Central Toronto'].reset_index(drop=True)
CentralToronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
1,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
2,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
3,M4S,Central Toronto,[Davisville],43.704324,-79.388790
4,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160


In [37]:
address = 'Central Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6449033, -79.3818364.


In [38]:
# create map of Manhattan using latitude and longitude values
map_CentralToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(CentralToronto_data['Latitude'], CentralToronto_data['Longitude'], CentralToronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CentralToronto)  
    
map_CentralToronto

## Adding Foursquare Credentials
Note: I have taken out my own credentials. Please add your own.

In [39]:
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
CLIENT_ID = '04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323' # your Foursquare ID
CLIENT_SECRET = 'JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323
CLIENT_SECRET:JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB


Looking at the first neighbourhood in the list

In [40]:
CentralToronto_data.loc[0, 'Neighbourhood']

['Lawrence Park']

Get the geo data for the neighbourhood 

In [41]:
neighborhood_latitude = CentralToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CentralToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = CentralToronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Lawrence Park'] are 43.7280205, -79.3887901.


## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

### Get the top 100 venues within a radius of 500 m

In [42]:
radius = 500
LIMIT = 100

In [69]:
# define URL
#url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

# send GET request and get trending venues
results = requests.get(url).json()
#results
'There are {} venues around Central Toronto (because I limited the request).'.format(len(results['response']['groups'][0]['items']))

'There are 100 venues around Central Toronto (because I limited the request).'

#### Process JSON and convert it to a clean dataframe

In [70]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '55561f8a498e2e5c866ac4c9',
  'name': 'Union Pearson Express',
  'location': {'address': '61 Front St. W',
   'lat': 43.64436200658875,
   'lng': -79.38319927907953,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64436200658875,
     'lng': -79.38319927907953}],
   'distance': 125,
   'postalCode': 'M5J 2Z8',
   'cc': 'CA',
   'neighborhood': 'Entertainment District',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['61 Front St. W', 'Toronto ON M5J 2Z8', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d129951735',
    'name': 'Train Station',
    'pluralName': 'Train Stations',
    'shortName': 'Train Station',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups'

In [73]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [74]:


dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Union Pearson Express,Train Station,61 Front St. W,43.644362,-79.383199,"[{'label': 'display', 'lat': 43.64436200658875...",125,M5J 2Z8,CA,Entertainment District,Toronto,ON,Canada,"[61 Front St. W, Toronto ON M5J 2Z8, Canada]",NaN,55561f8a498e2e5c866ac4c9
1,WVRST,Beer Bar,65 Front St W,43.644968,-79.381376,"[{'label': 'display', 'lat': 43.64496809087762...",37,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W, Toronto ON M5J 1E6, Canada]",NaN,5c08474447f876002c736b68
2,Pilot Coffee Roasters,Coffee Shop,65 Front St W,43.645018,-79.380415,"[{'label': 'display', 'lat': 43.64501814464698...",115,M5J 1E6,CA,NaN,Toronto,ON,Canada,"[65 Front St W (btwn Bay St & York St), Toront...",btwn Bay St & York St,563d2f2dcd10bcf27ae37c3b
3,iQ Food Co,Salad Place,18 York Street,43.642851,-79.382081,"[{'label': 'display', 'lat': 43.642851, 'lng':...",229,M5J 0B2,CA,NaN,Toronto,ON,Canada,"[18 York Street (Bremner Ave), Toronto ON M5J ...",Bremner Ave,5346c98a498ed612110d0f60
4,The Fairmont Royal York,Hotel,100 Front St W,43.645449,-79.381508,"[{'label': 'display', 'lat': 43.64544914616651...",66,M5J 1E3,CA,NaN,Toronto,ON,Canada,"[100 Front St W (btwn York St & Bay St), Toron...",btwn York St & Bay St,4ad4c05bf964a520a7f520e3
5,Union Chicken,Fried Chicken Joint,NaN,43.644912,-79.382325,"[{'label': 'display', 'lat': 43.64491185903613...",39,M5J 2Z1,CA,NaN,Toronto,ON,Canada,"[Toronto ON M5J 2Z1, Canada]",NaN,5a37fcbf123a1968dcbc4fe2
6,Real Sports Apparel,Sporting Goods Shop,15 York St. Unit B,43.642860,-79.380184,"[{'label': 'display', 'lat': 43.64285984835777...",263,M5J,CA,NaN,Toronto,ON,Canada,"[15 York St. Unit B (Maple Leaf Square), Toron...",Maple Leaf Square,4be46c832468c92887c1fe42
7,Maple Leaf Square,Plaza,15 York St.,43.642925,-79.380892,"[{'label': 'display', 'lat': 43.64292522840183...",232,NaN,CA,NaN,Toronto,ON,Canada,"[15 York St. (Bremner Blvd.), Toronto ON, Canada]",Bremner Blvd.,4bdb8c1cc79cc928a77583e9
8,Balzac's Coffee,Coffee Shop,7 Station Street (inside UP Express terminal),43.644373,-79.383065,"[{'label': 'display', 'lat': 43.64437258414836...",115,M5J 1C3,CA,NaN,Toronto,ON,Canada,[7 Station Street (inside UP Express terminal)...,The PATH - Skywalk,557f05ca498ec78ac7b29315
9,Delta Hotels by Marriott Toronto,Hotel,75 Lower Simcoe Street,43.642882,-79.383949,"[{'label': 'display', 'lat': 43.6428819, 'lng'...",282,M5J 3A6,CA,NaN,Toronto,ON,Canada,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, C...",NaN,53357710498e20817350cfb4


In [75]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Central Toronto',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

In [62]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d676f56cad1b6002c0df185'},
 'response': {'venues': []}}

### Check for trending venues

In [66]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [67]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

### Visualize trending venues

In [68]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)